# Bag of Words Feature Extraction

In this notebook we are going to extract Bag of Words (BoW) features from a dataset of image patches that correspond to either text or non-text areas in an image.

The roadmap is as follows:

* Import train/test data (raw image pixels). We use the same dataset as in PR1, raw_pixels_dataset_5980.pklz
* Extract small image patches from your training set
* Use K-Means to learn a vocabulary of Visual Words
* Represent the train images as BoW histograms
* Represent the test images as BoW histograms

### Usupervised Learning of Visual Words

In [ ]:
# Read the Dataset (Raw Pixels data)
import pickle
import gzip
import numpy as np
import matplotlib.pyplot as plt

with gzip.open('./raw_pixels_dataset_5980.pklz','rb') as f:
    (train_labels,train_images,test_labels,test_images) = pickle.load(f, encoding='latin1')

print("Train_images shape " + str(train_images.shape))
print("Test_images shape  " + str(test_images.shape))

# Show a few samples of the positive and negative classes.
num_text = sum(train_labels==0)
fig = plt.figure()
for i in range(1,6):
    ax = fig.add_subplot(2, 5, i)
    ax.imshow(np.reshape(train_images[np.random.randint(0,num_text),:],[32,32]), cmap=plt.cm.gray)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax = fig.add_subplot(2, 5, i+5)
    ax.imshow(np.reshape(train_images[np.random.randint(num_text,train_labels.shape[0]),:],[32,32]), cmap=plt.cm.gray)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)


In [ ]:
# Extract patches (of 8x8 pixels) from train images
import numpy as np

images = train_images.reshape((-1,32,32)).astype('float32')/255 # reshape images to 32x32
                                                                # and scale values to (0., 1.)
print("Images shape " + str(images.shape))

# Collect image patches with sliding window (8x8) in each train image sample
PATCH_SIZE=8;
STEP_SIZE =8;

patches = np.zeros((0,PATCH_SIZE,PATCH_SIZE))

for x in range(0, 32-PATCH_SIZE+1, STEP_SIZE):
    for y in range(0, 32-PATCH_SIZE+1, STEP_SIZE):
        patches = np.concatenate((patches, images[:,x:x+PATCH_SIZE,y:y+PATCH_SIZE]), axis=0)

patches = patches.reshape((patches.shape[0],-1))
print("Patches shape " + str(patches.shape))

# Visualize a few patches
fig = plt.figure()
for i in range(1,6):
    ax = fig.add_subplot(1, 5, i)
    ax.imshow(np.reshape(patches[i],[PATCH_SIZE,PATCH_SIZE]), cmap=plt.cm.gray, vmin=0, vmax=1)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

Now we contrast normalize our patches in order to gain illumination invariance. Each patch is normalized by pixel-wise subtracting its mean and dividing by its standard deviation.

In [ ]:
# PreProcessing (I) Contrast Normalization of Patches

mu = patches.mean(axis=1) # mean values
sigma = patches.std(axis=1) + np.ptp(patches, axis=1)/20.0 # standard deviation (plus a small value)

patches = (patches-mu.reshape([-1,1]))/(sigma.reshape([-1,1])) # subtract the mean and divide by std

# Set NaN values (if exist) to 0
w = np.isnan(patches);
patches[w] = 0;

# Visualize a few Constrast Normalized patches
fig = plt.figure()
for i in range(1,6):
    ax = fig.add_subplot(1, 5, i)
    ax.imshow(np.reshape(patches[i],[PATCH_SIZE,PATCH_SIZE]), cmap=plt.cm.gray)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

Next we perform ZCA whitening of the patches. 

We know how to use PCA to reduce the dimensionality of the data. ZCA whitening is a closely related preprocessing step. 

When we are working with images the raw input is redundant, because the values of adjacent pixels in an image are highly correlated. The goal of whitening is to decorrelate pixel values with each other and make them all have the same variance.

ZCA whitening is implemented as follows:

* First, subtract the mean value of each pixel in image patches to make our data zero-mean.
* Then, as in PCA, compute the eigenvectors of the patches' covariance matrix $\Sigma$.
* The ZCA whitened data is then : 

$x_{ZCAwhite} = U * diag(1./\sqrt{diag(S) + \epsilon)} * U' * x$

where $U$ contains the eigenvectors of $\Sigma$, $S$ contains the corresponding eigenvalues, $\epsilon$ is a trivial small value and $x$ is the input data (image patches).

More info: http://ufldl.stanford.edu/wiki/index.php/Whitening

In [ ]:
# PreProcessing (II) ZCA Whitening of normalized patches

eig_values, eig_vec = np.linalg.eig(np.cov(patches.T))
epsilon = 0.01
pca = eig_vec.dot(np.diag((eig_values+epsilon)**-0.5).dot(eig_vec.T))


M =  patches.mean(axis=0)
patches = patches -  M # subtract average value
patches = np.dot(patches, pca) # perform pca whitening


# Visualize a few PreProcessed patches
fig = plt.figure()
for i in range(1,6):
    ax = fig.add_subplot(1, 5, i)
    ax.imshow(np.reshape(patches[i],[PATCH_SIZE,PATCH_SIZE]), cmap=plt.cm.gray)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

Once we have our preprocessed training patches we can perform Unsupervised Learning analysis (K-means clustering) to learn the K visual Words vocabulary.  Then we'll use these vocabulary to represent an image as an histogram of the occurrences of its Words.

In [ ]:
# K-means clustering to learn K visual_words from data
from sklearn.cluster import KMeans

NUM_VISUAL_WORDS = 64

km = KMeans(n_clusters=NUM_VISUAL_WORDS, max_iter=50, n_init=1, verbose=False)
km.fit(patches)

visual_words = km.cluster_centers_
print("Visual_words shape " + str(visual_words.shape))

In [ ]:
# Visualize the learned vocabulary of Visual Words

from matplotlib import pyplot as plt
import matplotlib.cm as cm
#%pylab inline

fig = plt.figure()
num_col = int(np.ceil(float(NUM_VISUAL_WORDS)/4))
for i in range(NUM_VISUAL_WORDS):
    ax = fig.add_subplot(4, num_col, i+1)
    visual_word_ = visual_words[i,:]
    visual_word_ = visual_word_.reshape(PATCH_SIZE,PATCH_SIZE);
    ax.imshow(visual_word_, interpolation='none', cmap = cm.Greys_r)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

### Extract Image features using Bag of Words

A Bag of Visual Words is a histogram of occurrence counts of the vocabulary words.

In our case we have 64 Words in the vocabulary, hence the histogram will have 64 bins. 

To represent an image we'll extract image patches, preprocess them, and find its Nearest Neighbor Word in the vocabulary. Finally, each Nearest Neighbor found increments in 1 unit the value of its respective bin in the BoW histogram. 

In [ ]:
# Learn a KNN classifier, Each visual Word represents one class
# In this case KNN is used simply as a way to search of the nearest neighbor Visual Word 
# in our vocabulary.
from sklearn.neighbors import KNeighborsClassifier

neig = KNeighborsClassifier(n_neighbors=1)
neig.fit(visual_words,range(0,NUM_VISUAL_WORDS));

In [ ]:
# Extract features from train images
from tqdm import tqdm

train_features = np.zeros((images.shape[0],visual_words.shape[0]));

for i in tqdm(range(0,images.shape[0])): # for each image

  # Do sliding window (8x8) in each image to extract patches
  # then normalize, whiten and build the Bag of Words histogram
    for x in range(0,32-PATCH_SIZE+1,STEP_SIZE):
        for y in range(0,32-PATCH_SIZE+1,STEP_SIZE):
            patch = images[i,x:x+PATCH_SIZE,y:y+PATCH_SIZE]
            patch = patch.reshape((1,-1))
            # PreProcessing (I): Normalize
            mu = patch.mean(axis=1) # mean values
            sigma = patch.std(axis=1) + max(np.ptp(patch, axis=1)/20.0, 0.0001) # standard deviation 
                                                                   # (plus a small value)
            patch = (patch-(mu[np.newaxis,:]).T)/(sigma[np.newaxis,:]).T
            # Set NaN values to 0
            w = np.isnan(patch);
            patch[w] = 0;

            # PreProcessing (II): ZCA whitening
            patch = patch - M # subtract average value
            patch = np.dot(patch, pca) # perform pca whitening

            # BoW
            nn = neig.predict(patch)
            train_features[i,nn] = train_features[i,nn] + 1;
        
    train_features[i,:] = train_features[i,:]/max(train_features[i,:]) # Histogram normalization

In [ ]:
# If you want to save your training features
import pickle
import gzip

with gzip.open('./BoW_train_features.pklz','wb') as f:
    pickle.dump((train_labels,train_features),f,pickle.HIGHEST_PROTOCOL)

In [ ]:
# Extract features from test images
images = test_images.reshape((-1,32,32)).astype('float32')/255
test_features = np.zeros((images.shape[0],visual_words.shape[0]));

for i in tqdm(range(0,images.shape[0])): #for each image
    
  # Do sliding window (8x8) in each image to extract patches
  # then normalize, whiten and build the Bag of Words histogram
    for x in range(0,32-PATCH_SIZE+1,STEP_SIZE):
        for y in range(0,32-PATCH_SIZE+1,STEP_SIZE):
            patch = images[i,x:x+PATCH_SIZE,y:y+PATCH_SIZE]
            patch = patch.reshape((1,-1))
            # PreProcessing (I): Normalize
            mu = patch.mean(axis=1) # mean values
            sigma = patch.std(axis=1) + max(np.ptp(patch, axis=1)/20.0, 0.0001) # standard deviation 
                                                                   # (plus a small value)
            patch = (patch-(mu[np.newaxis,:]).T)/(sigma[np.newaxis,:]).T
            # Set NaN values to 0
            w = np.isnan(patch);
            patch[w] = 0;

            # PreProcessing (II): ZCA whitening
            patch = patch - M # subtract average value
            patch = np.dot(patch, pca) # perform pca whitening

            # BoW
            nn = neig.predict(patch)
            test_features[i,nn] = test_features[i,nn] + 1;
        
    test_features[i,:] = test_features[i,:]/max(test_features[i,:])  # Histogram normalization

In [ ]:
# If you want to save your test features
import pickle
import gzip

with gzip.open('./BoW_test_features.pklz','wb') as f:
    pickle.dump((test_labels,test_features),f,pickle.HIGHEST_PROTOCOL)